<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/Alpha_Beta_Crown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# α,β-CROWN ML-Verification Tutorial

## Installation

Installation by [tutorial](https://colab.research.google.com/drive/1mJTOmq2qHxMycHUzBepBN47QWcxda3ov#scrollTo=Y0toepwVIFTG). 

In [1]:
%%capture
%%bash
%env PYTHONPATH=
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [2]:
%%capture
%%bash
conda install --channel defaults conda python=3.7 --yes
conda update --channel defaults --all --yes

In [3]:
%%capture
import sys
sys.path
!ls /usr/local/lib/python3.7/dist-packages
_ = (sys.path
        .append("/usr/local/lib/python3.7/site-packages"))

In [4]:
%%capture
# Uninstall existing Pytorch on Colab, which might be incompatible or buggy.
# !pip uninstall --yes torch torchvision torchaudio torchtext
!git clone https://github.com/huanzhang12/alpha-beta-CROWN.git

In [5]:
%%capture
%%bash
# Remove the old environment, if necessary.
conda env remove --name alpha-beta-crown
conda env create -f alpha-beta-CROWN/complete_verifier/environment.yml  # install all dependents into the alpha-beta-crown environment
#conda activate alpha-beta-crown  # activate the environment

In [6]:
%cd alpha-beta-CROWN/complete_verifier/

/content/alpha-beta-CROWN/complete_verifier


In [28]:
%%writefile -a exp_configs/tutorial_mnist_example.yaml
general:
  mode: verified-acc
model:
  name: mnist_cnn_4layer
  path: models/sdp/mnist_cnn_a_adv.model
data:
  dataset: MNIST
  std: [1.]
  mean: [0.]
specification:
  epsilon: 0.3
attack:
  pgd_restarts: 50
solver:
  beta-crown:
    batch_size: 1024
    iteration: 20
bab:
  timeout: 180

Writing exp_configs/tutorial_mnist_example.yaml


## Verification of sample

In [29]:
%%bash
source activate alpha-beta-crown
# python robustness_verifier.py --config exp_configs/tutorial_mnist_example.yaml --start 0 --end 1
# python robustness_verifier.py --config exp_configs/tutorial_mnist_example.yaml --start 1 --end 2
python robustness_verifier.py --config exp_configs/tutorial_mnist_example.yaml --start 21 --end 22
conda deactivate

Configurations:

general:
  device: cuda
  seed: 100
  conv_mode: patches
  deterministic: false
  double_fp: false
  loss_reduction_func: sum
  record_bounds: false
  mode: verified-acc
  complete_verifier: bab
  enable_incomplete_verification: true
  get_crown_verified_acc: false
model:
  path: models/sdp/mnist_cnn_a_adv.model
  name: mnist_cnn_4layer
data:
  start: 21
  end: 22
  num_outputs: 10
  mean: [0.0]
  std: [1.0]
  pkl_path: null
  dataset: MNIST
  data_filter_path: null
  data_idx_file: null
specification:
  type: lp
  norm: .inf
  epsilon: 0.3
solver:
  alpha-crown:
    lr_alpha: 0.1
    iteration: 100
    share_slopes: false
    no_joint_opt: false
  beta-crown:
    batch_size: 1024
    lr_alpha: 0.01
    lr_beta: 0.05
    lr_decay: 0.98
    optimizer: adam
    iteration: 20
    beta: true
    beta_warmup: true
  mip:
    parallel_solvers: null
    solver_threads: 1
    refine_neuron_timeout: 15
    refine_neuron_time_percentage: 0.8
    early_stop: true
bab:
  max_domai

9913344it [00:00, 81916967.88it/s]                             
29696it [00:00, 59652323.56it/s]         
1649664it [00:00, 24325323.49it/s]         
5120it [00:00, 20668755.03it/s]         
/usr/local/envs/alpha-beta-crown/lib/python3.7/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1627336316785/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/envs/alpha-beta-crown/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker pro

## Training of own network

TODO: train network

## Verification of own network with α,β-CROWN

TODO: implement